In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
!pip install pyTigerGraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.0 MB/s eta 0:00:00


In [14]:
import pandas as pd
import sqlite3
import time
import pyTigerGraph as tg
from google.colab import drive
import concurrent.futures
import psutil  # For resource usage monitoring

# Mount Google Drive
drive.mount('/content/drive')

# Load CSV files into pandas DataFrames from Google Drive
df1 = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/medications_csv.csv')
df2 = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/allergies_csv.csv')
df3 = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/patients_csv.csv')
df4 = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/conditions.csv')

# Print column names to verify
# print("Columns in df1:", df1.columns)
# print("Columns in df2:", df2.columns)
# print("Columns in df3:", df3.columns)
# print("Columns in df4:", df4.columns)

# Create a connection to an in-memory SQLite database
sql_conn = sqlite3.connect(':memory:')

# Function to measure load time for SQL
def measure_sql_load_time():
    start_time = time.time()
    df1.to_sql('medications', sql_conn, index=False, if_exists='replace')
    df2.to_sql('allergies', sql_conn, index=False, if_exists='replace')
    df3.to_sql('patients', sql_conn, index=False, if_exists='replace')
    df4.to_sql('conditions', sql_conn, index=False, if_exists='replace')
    end_time = time.time()
    return end_time - start_time

# Write the dataframes to the SQLite database with appropriate table names and column mapping
sql_load_time = measure_sql_load_time()

# Connect to TigerGraph
hostname = "https://5c97db8ab1d345a499a689ead868baa6.i.tgcloud.io"
graphName = "clinical_graph"
secret = "oa42tb3jfp6ntubbr7qlnt34gccsfprn"
username = "ashma_garg"
password = "Clinical@1"

try:
    graph = tg.TigerGraphConnection(host=hostname, graphname=graphName, username=username, password=password, gsqlSecret=secret)
    authToken = graph.getToken(secret)
except Exception as e:
    print(f"Error connecting to TigerGraph: {e}")
    graph = None

# Function to measure load time for TigerGraph
def measure_tg_load_time(graph, df1, df2, df3, df4):
    start_time = time.time()
    try:
        # Define loading jobs
        df1.to_csv('/content/medications.csv', index=False)
        df2.to_csv('/content/allergies.csv', index=False)
        df3.to_csv('/content/patients.csv', index=False)
        df4.to_csv('/content/conditions.csv', index=False)

        # Upload data to TigerGraph
        graph.uploadFile('/content/medications.csv', 'medications_csv', 'medications', separator=',')
        graph.uploadFile('/content/allergies.csv', 'allergies_csv', 'allergies', separator=',')
        graph.uploadFile('/content/patients.csv', 'patients_csv', 'patients', separator=',')
        graph.uploadFile('/content/conditions_csv.csv', 'conditions_csv', 'conditions', separator=',')

        # Run loading job
        load_job = """
        DEFINE FILENAME medications_csv = "/content/medications.csv";
        DEFINE FILENAME allergies_csv = "/content/allergies.csv";
        DEFINE FILENAME patients_csv = "/content/patients.csv";
        DEFINE FILENAME conditions_csv = "/content/conditions_csv.csv";

        LOAD medications_csv TO VERTEX medications VALUES($0, $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12, $13) USING header="true", separator=",";
        LOAD allergies_csv TO VERTEX allergies VALUES($0, $1, $2, $3, $4, $5, $6) USING header="true", separator=",";
        LOAD patients_csv TO VERTEX patients VALUES($0, $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12, $13, $14, $15, $16, $17, $18, $19, $20, $21, $22, $23) USING header="true", separator=",";
        LOAD conditions_csv TO VERTEX conditions VALUES($0, $1, $2, $3, $4, $5) USING header="true", separator=",";
        """
        graph.gsql(load_job)
    except Exception as e:
        #print(f"Error loading data into TigerGraph: {e}")
        pass
    end_time = time.time()
    return end_time - start_time

# Measure TigerGraph load time
tg_load_time = measure_tg_load_time(graph, df1, df2, df3, df4) if graph else "N/A"

# Function to measure execution time of a SQL query
def measure_sql_query_time(query, conn):
    start_time = time.time()
    result = pd.read_sql_query(query, conn)
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time, result

# Function to measure execution time of a GSQL query
def measure_gsql_query_time(graph, query):
    if graph:
        start_time = time.time()
        try:
            result = graph.runInterpretedQuery(query)
            end_time = time.time()
            execution_time = end_time - start_time
            return execution_time, result
        except Exception as e:
            print(f"Error executing query: {e}")
            return None, None
    return None, None

# Function to measure throughput for SQL queries
def measure_sql_throughput(queries, conn):
    def run_query(query):
        measure_sql_query_time(query, conn)

    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(run_query, query) for query in queries]
        concurrent.futures.wait(futures)
    end_time = time.time()
    total_time = end_time - start_time
    throughput = len(queries) / total_time
    return throughput

# Function to measure throughput for TigerGraph queries
def measure_gsql_throughput(graph, queries):
    def run_query(query):
        measure_gsql_query_time(graph, query)

    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(run_query, query) for query in queries]
        concurrent.futures.wait(futures)
    end_time = time.time()
    total_time = end_time - start_time
    throughput = len(queries) / total_time
    return throughput

# Function to monitor resource usage
def monitor_resource_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    memory_usage = psutil.virtual_memory().percent
    disk_usage = psutil.disk_usage('/').percent
    return cpu_usage, memory_usage, disk_usage

# Benchmarking SQL queries
sql_queries = {
    "Get Patient Allergies": '''
        SELECT patients.Id, allergies.DESCRIPTION
        FROM patients
        JOIN allergies ON patients.Id = allergies.PATIENT
    ''',
    "Maximum healthcare cost difference": '''
        SELECT *, (HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) AS health_cost
        FROM patients
        WHERE (HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) = (
            SELECT MAX(HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE)
            FROM patients
        );
    ''',
    "Medications to Related Conditions": '''
        SELECT c.*
        FROM medications m
        JOIN conditions c ON c.CODE = m.REASONCODE
        WHERE m.CODE = '1043400';
    ''',

       "Average Healthcare expenses" : '''
       SELECT AVG(HEALTHCARE_EXPENSES) AS avg_healthcare_expenses
       FROM patients;
       ''',
    "Maximum city healthcare cost": '''
     WITH CityPatients AS (
    SELECT
        (HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) AS expense_diff
    FROM
        patients
    WHERE
        UPPER(CITY) = UPPER('Franklin')
    ),
   MaxExpense AS (
    SELECT
        MAX(expense_diff) AS max_expense_diff
    FROM
        CityPatients
   )
   SELECT
    p.*,
    (p.HEALTHCARE_EXPENSES - p.HEALTHCARE_COVERAGE) AS expense_diff
    FROM
    patients p
    JOIN
    CityPatients cp ON (p.HEALTHCARE_EXPENSES - p.HEALTHCARE_COVERAGE) = cp.expense_diff
    WHERE
    UPPER(p.CITY) = UPPER('Franklin');
    ''',

    "Country wise highest expense from patient" : '''
     WITH CountryPatients AS (
    SELECT
        (HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) AS expense_diff
    FROM
        patients
    WHERE
        UPPER(COUNTY) = UPPER('Essex County')
),
MaxExpense AS (
    SELECT
        MAX(expense_diff) AS max_expense_diff
    FROM
        CountryPatients
)
SELECT
    p.*,
    (p.HEALTHCARE_EXPENSES - p.HEALTHCARE_COVERAGE) AS expense_diff
FROM
    patients p
JOIN
    CountryPatients cp ON (p.HEALTHCARE_EXPENSES - p.HEALTHCARE_COVERAGE) = cp.expense_diff
WHERE
    UPPER(p.COUNTY) = UPPER('Essex County');
    '''
    }

sql_times = {}
for query_name, query in sql_queries.items():
    execution_time, result = measure_sql_query_time(query, sql_conn)
    sql_times[query_name] = execution_time
    # if query_name == "Medications to Related Conditions":
        # print(f"Result for {query_name}:\n", result)

# Benchmarking TigerGraph queries
gsql_queries = {
    "Get Patient Allergies": """
        INTERPRET QUERY getPatientAllergies() FOR GRAPH clinical_graph {
            result = SELECT p FROM patients:p - (allergy_patient:e) -> allergies:a;
            PRINT result;
        }""",
    "Maximum healthcare cost difference": """
        INTERPRET QUERY max_healthcare_cost() FOR GRAPH clinical_graph {
            MaxAccum<FLOAT> @@max_healthcost;

            graph_set = {patients.*};

            patients_data = SELECT p FROM graph_set:p
                            ACCUM
                                @@max_healthcost += (p.healthcare_expenses - p.healthcare_coverage);
            max_expense_patient_data = SELECT p from graph_set:p
                                        where abs(((p.healthcare_expenses - p.healthcare_coverage)) - (@@max_healthcost)) < 0.000000001;
            PRINT @@max_healthcost;
            PRINT max_expense_patient_data;
        }""",
    "Medications to Related Conditions": """
        INTERPRET QUERY q1__med_to_condition_title (STRING code = "1043400") for graph clinical_graph {
            Seed = {medication.*};
            medic = SELECT s FROM Seed:s WHERE s.medication_code == code;
            med_related_conditions = SELECT c FROM conditions:c - (condition_medication:e) - medication:m WHERE m.medication_code == code;
            PRINT medic;
            PRINT med_related_conditions;
        }""",

  "Average Healthcare expenses": """
        INTERPRET QUERY avg_hc() FOR GRAPH clinical_graph {
            AvgAccum @@avg_healthcost;

            graph_set = {patients.*};

            patients_data = SELECT p FROM graph_set:p
                            ACCUM
                                @@avg_healthcost += p.healthcare_expenses;
            PRINT @@avg_healthcost;
        }""",

   "Maximum city healthcare cost": """
    INTERPRET QUERY max_city_healthcare_cost(STRING city = "Franklin") FOR GRAPH clinical_graph {
        MaxAccum<FLOAT> @@city_max_healthcost;

        graph_set = {patients.*};

        patients_data = SELECT p FROM graph_set:p where UPPER(p.city) == UPPER(city)
                        ACCUM
                            @@city_max_healthcost += (p.healthcare_expenses - p.healthcare_coverage);
        max_expense_patient_data = SELECT p from graph_set:p
                                    where abs(((p.healthcare_expenses - p.healthcare_coverage)) - (@@city_max_healthcost)) < 0.000000001;
        PRINT @@city_max_healthcost;
        PRINT max_expense_patient_data;
    }""",

    "Country wise highest expense from patient" :"""
    INTERPRET QUERY max_country_healthcare_cost(STRING country="Essex County") FOR GRAPH clinical_graph {
        MaxAccum<FLOAT> @@country_max_healthcost;

        graph_set = {patients.*};

        patients_data = SELECT p FROM graph_set:p where UPPER(p.country) == UPPER(country)
                        ACCUM
                            @@country_max_healthcost += (p.healthcare_expenses - p.healthcare_coverage);
        max_expense_patient_data = SELECT p from graph_set:p
                                    where abs(((p.healthcare_expenses - p.healthcare_coverage)) - (@@country_max_healthcost)) < 0.000000001;
        PRINT @@country_max_healthcost;
        PRINT max_expense_patient_data;
    }"""
}

tg_times = {}
if graph:
    for query_name, query in gsql_queries.items():
        execution_time, result = measure_gsql_query_time(graph, query)
        tg_times[query_name] = execution_time
        # if query_name == "Medications to Related Conditions":
        #     print(f"Result for {query_name}:\n", result)
else:
    for query_name in gsql_queries:
        tg_times[query_name] = None

# Measure SQL throughput
sql_queries_list = list(sql_queries.values())
sql_throughput = measure_sql_throughput(sql_queries_list, sql_conn)

# Measure TigerGraph throughput (for demonstration purposes)
tg_queries_list = list(gsql_queries.values())
tg_throughput = measure_gsql_throughput(graph, tg_queries_list) if graph else "N/A"

# Monitor resource usage
cpu, memory, disk = monitor_resource_usage()

# Print benchmarking results
print("\nBenchmarking Results Comparison:")

print("\nSQL Database Metrics:")
print(f"Load Time: {sql_load_time:.4f} seconds")
print(f"Throughput: {sql_throughput:.2f} queries/second")
print(f"Resource Usage - CPU: {cpu}%, Memory: {memory}%, Disk: {disk}%")

print("\nTigerGraph Metrics:")
print(f"Load Time: {tg_load_time if tg_load_time != 'N/A' else 'N/A'} seconds")
print(f"Throughput: {tg_throughput if tg_throughput != 'N/A' else 'N/A'} queries/second")
print(f"Resource Usage - CPU: {cpu}%, Memory: {memory}%, Disk: {disk}%")

print("\nQuery Execution Times:")
print("SQL Queries:")
for query, time_taken in sql_times.items():
    print(f"{query}: {time_taken:.4f} seconds")

print("\nTigerGraph Queries:")
if graph:
    for query, time_taken in tg_times.items():
        if time_taken is not None:
            print(f"{query}: {time_taken:.4f} seconds")
        else:
            print(f"{query}: Unable to connect to TigerGraph")
else:
    print("Unable to connect to TigerGraph")

print("\nPerformance Comparison:")
for query in sql_times:
    tg_time = tg_times.get(query, "N/A")
    sql_time = sql_times[query]
    print(f"{query}: TigerGraph = {tg_time if tg_time != 'N/A' else 'N/A'} seconds, SQL = {sql_time:.4f} seconds")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Benchmarking Results Comparison:

SQL Database Metrics:
Load Time: 0.3486 seconds
Throughput: 3966.24 queries/second
Resource Usage - CPU: 3.0%, Memory: 12.4%, Disk: 25.0%

TigerGraph Metrics:
Load Time: 0.5468919277191162 seconds
Throughput: 9.308278307853689 queries/second
Resource Usage - CPU: 3.0%, Memory: 12.4%, Disk: 25.0%

Query Execution Times:
SQL Queries:
Get Patient Allergies: 0.0036 seconds
Maximum healthcare cost difference: 0.0025 seconds
Medications to Related Conditions: 0.4162 seconds
Average Healthcare expenses: 0.0015 seconds
Maximum city healthcare cost: 0.0053 seconds
Country wise highest expense from patient: 0.0402 seconds

TigerGraph Queries:
Get Patient Allergies: 0.4006 seconds
Maximum healthcare cost difference: 0.2899 seconds
Medications to Related Conditions: 0.2566 seconds
Average Healthcare expenses: 0.2545 seconds
Maximum city

In [9]:
!pip install pandas

In [12]:
import pandas as pd
import sqlite3
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Load CSV files into pandas DataFrames
df_medications = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/medications_csv.csv')
df_allergies = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/allergies_csv.csv')
df_patients = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/patients_csv.csv')
df_conditions = pd.read_csv('/content/drive/MyDrive/ADT_PROJECT/conditions.csv')

# Print column names to verify, just to make sure about the tables we are working
# print("Columns in df_medications:", df_medications.columns)
# print("Columns in df_allergies:", df_allergies.columns)
# print("Columns in df_patients:", df_patients.columns)
# print("Columns in df_conditions:", df_conditions.columns)

# Step 3: Create an SQLite database in memory
conn = sqlite3.connect(':memory:')

# Step 4: Write the DataFrames to SQLite tables
df_medications.to_sql('Medications', conn, index=False, if_exists='replace')
df_allergies.to_sql('Allergies', conn, index=False, if_exists='replace')
df_patients.to_sql('Patients', conn, index=False, if_exists='replace')
df_conditions.to_sql('Conditions', conn, index=False, if_exists='replace')

# Step 5: Function to execute a query and print the results
def execute_query(query):
    print("Executing query:")
    print(query)
    try:
        result = pd.read_sql_query(query, conn)
        print(result)
    except Exception as e:
        print("Error:", e)
    print("-------------------------------------------------------------------------------------------")

# Example Queries, these are the exactly one which we ran on the tiger graph

# Test Query, just to check!
execute_query("SELECT 'Test Query 1 Works!' AS message;")

# Conditions related to medication by code
execute_query("""
    SELECT DISTINCT c.*
    FROM Conditions c
    JOIN Medications m ON c.PATIENT = m.PATIENT
    WHERE m.CODE = '1043400';
""")

# Medications related to a condition by code
execute_query("""
    SELECT DISTINCT m.*
    FROM Medications m
    JOIN Conditions c ON c.PATIENT = m.PATIENT
    WHERE c.CODE = '10509002';
""")

# Conditions and medications by condition description
execute_query("""
    SELECT DISTINCT c.*
    FROM Conditions c
    WHERE UPPER(c.DESCRIPTION) LIKE UPPER('%Acute bron%');
""")

execute_query("""
    SELECT DISTINCT m.*
    FROM Medications m
    JOIN Conditions c ON c.PATIENT = m.PATIENT
    WHERE UPPER(c.DESCRIPTION) LIKE UPPER('%Acute bron%');
""")

# Average healthcare expenses
execute_query("""
    SELECT AVG(HEALTHCARE_EXPENSES) AS avg_healthcost
    FROM Patients;
""")

# Highest healthcare expense
execute_query("""
    SELECT MAX(HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) AS max_healthcost
    FROM Patients;
""")

# Highest healthcare expense by city
execute_query("""
    SELECT MAX(HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) AS city_max_healthcost
    FROM Patients
    WHERE UPPER(CITY) = UPPER('Franklin');
""")

# Highest healthcare expense by county
execute_query("""
    SELECT MAX(HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) AS county_max_healthcost
    FROM Patients
    WHERE UPPER(COUNTY) = UPPER('Essex County');
""")

# Highest healthcare expense by state
execute_query("""
    SELECT MAX(HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) AS state_max_healthcost
    FROM Patients
    WHERE UPPER(STATE) = UPPER('Massachusetts');
""")

# Patients paying more than threshold or average healthcare expenses
threshold = 1500000  # Example threshold value
execute_query(f"""
    WITH avg_healthcost AS (
        SELECT AVG(HEALTHCARE_EXPENSES) AS avg_cost
        FROM Patients
    )
    SELECT COUNT(*) AS patient_paying_above_threshold_count
    FROM Patients, avg_healthcost
    WHERE (HEALTHCARE_EXPENSES - HEALTHCARE_COVERAGE) > {threshold};
""")

# Most common condition
execute_query("""
    WITH condition_counts AS (
        SELECT CODE, COUNT(*) AS count
        FROM Conditions
        JOIN Patients ON Conditions.PATIENT = Patients.Id
        GROUP BY CODE
    )
    SELECT CODE, MAX(count) AS max_count
    FROM condition_counts
""")

# Most common condition in a specific city
execute_query("""
    WITH city_conditions AS (
        SELECT CODE, COUNT(*) AS count
        FROM Conditions
        JOIN Patients ON Conditions.PATIENT = Patients.Id
        WHERE UPPER(Patients.CITY) = UPPER('Chatham')
        GROUP BY CODE
    )
    SELECT CODE, MAX(count) AS max_count
    FROM city_conditions
""")

# Condition with maximum duration
execute_query("""
    WITH condition_durations AS (
        SELECT c.Id AS condition_id,
               (CASE WHEN c.STOP = '' THEN JULIANDAY('now') ELSE JULIANDAY(c.STOP) END -
                CASE WHEN c.START = '' THEN JULIANDAY('now') ELSE JULIANDAY(c.START) END) AS duration
        FROM Conditions c
    )
    SELECT condition_id, MAX(duration) AS max_duration
    FROM condition_durations
""")

# Patients for a specific condition code
execute_query("""
    SELECT p.*
    FROM Patients p
    JOIN Conditions c ON p.Id = c.PATIENT
    WHERE c.CODE = '47693006';
""")

# Patients for a specific medication code
execute_query("""
    SELECT p.*
    FROM Patients p
    JOIN Medications m ON p.Id = m.PATIENT
    WHERE m.CODE = '310385';
""")

# Close the database connection
conn.close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Executing query:
SELECT 'Test Query 1 Works!' AS message;
               message
0  Test Query 1 Works!
-------------------------------------------------------------------------------------------
Executing query:

    SELECT DISTINCT c.*
    FROM Conditions c
    JOIN Medications m ON c.PATIENT = m.PATIENT
    WHERE m.CODE = '1043400';

        Id       START        STOP                               PATIENT  \
0       31  28-01-2012  11-02-2012  ad2e9916-4979-40fc-a8c0-68651a0cb5a6   
1       32  19-10-2017  26-10-2017  ad2e9916-4979-40fc-a8c0-68651a0cb5a6   
2       88  19-07-1977        None  86b97fc7-ae8f-4e0d-8e66-db68f36e7a76   
3       89  07-03-1995        None  86b97fc7-ae8f-4e0d-8e66-db68f36e7a76   
4       90  02-01-2007        None  86b97fc7-ae8f-4e0d-8e66-db68f36e7a76   
...    ...         ...         ...                                   ...   
